# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
#импортируем библиотеки pandas
import pandas as pd
#импортируем метод display из библиотеки IPython.display
from IPython.display import display
#импортируем библиотеку pymystem3 
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [2]:
#открываем файл с данными
data = pd.read_csv('/datasets/data.csv')

In [3]:
#рассмотрим таблицу с данными
display(data.head())
#получим общею информацию о данных в таблице data
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

По результатам изучения общей информации можно сделать несколько выводов:
1. Присутствуют пропуски в столбцах 'days_employed' и 'total_income'
2. Есть дубликаты
3. Используются не только целые числа, но и вещественные
4. Нет единого формата записи цели получения кредита
5. Общий трудовой стаж в днях имеет отрицательное значение

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
#заменим значения NaN в столбцах 'days_employed'
data['days_employed'] = data['days_employed'].fillna(0)
#определим медиану ежемесячного дохода
income_med = data['total_income'].median()
#заменим неизвестные значения в столбце 'total_income' значением медианы
data['total_income'] = data['total_income'].fillna(income_med)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

    В таблице найдены следующие пропущенные значения: в столбцах 'days_employed' и 'total_income' содержатся значения NaN.
    На появление пропусков возможно повлиял человеческий фактор или автоматизированная система сбора информации дала сбой.
    Т.к. данные об общем трудовом стаже в днях для нашей задачи не важны,но полное удаление строки может привести к потере нужных данных, заменяем все значения NaN в стобце 'days_employed' нулями. 
    Значения в столбце 'total_income' нам важны и заменить неизвестные значения нулями, будет не совсем правильно. Т.к. у нас нет возможности выяснить недостающие значения, мы можем использовать метод mean() или median. Выбираем метод median(), потому что среднее значение некорректно характеризует данные, когда некоторые значения сильно выделяются среди большинства.



### Замена типа данных

In [124]:
#заменим вещественный тип данных на целочисленный используя метод astype()
data[['days_employed','total_income']] = data[['days_employed','total_income']].astype('int')

**Вывод**

В столбцах 'days_employed' и 'total_income' числа имели тип float64. В нашем случае такой тип данных нам не подходит. Для перевода чисел в нужный тип воспользуемся методом astype(). Используя аргумент ('int') метода astype() переведем все значения в целые числа. 

### Работа с артифактами

In [125]:
#приведем числа в столбце 'days_employed' к абсолютному значению 
data['days_employed'] = data['days_employed'].abs()
#приведем числа в столбце 'days_employed' к абсолютному значению
data['children'] = data['children'].abs()
#изменим количество детей - 20, на верное значение - 2
data['children'] = data['children'].replace(20, 0)

**Вывод**

В столбцах 'days_employed' и 'children' мы видим, что числа имеют отрицательные значения. Это является ошибкой. Для ее устранения воспользуемся функцией abs() для получения модуля числа(возвращает абсолютное значение аргумента).
Также есть ошибка в указании количества детей. 20 детей в одной семье, это невозможный вариант. Данная ошибка могла возникнуть, например, в результате опечатки при вводе данных. Т.е. на правильность данных повлиял человеческий фактор. Исправим неправильное значение, применив метод replace.

### Обработка дубликатов

In [126]:
#определим дубликаты в столбце 'education'
duplicated_ed = data['education'].value_counts() 
#переведем все символы в столбце 'education' в нижний регистр
data['education'] = data['education'].str.lower()
#удалим явные дубликаты
data = data.drop_duplicates().reset_index(drop=True)
#найдем количесвто дубликатов
display(data.duplicated().sum())

0

**Вывод**

В таблице присутствовали как явные, так и неявные дубликаты. С помощью метода drop_duplicates() мы удалили все явные дубликаты.  Метод value_counts() позволил определить дубликаты в столбце 'education'. Данные дублировались из-за различных регистов в строках. Было принято решение привести все символы в строках к нижнему регистру. Это было сделано с помощью метода .str.lower(). Неявные дубликаты мы искали методом unique(). Для удаления найденных неявных дубликатов, воспользуемся лемматизацией, импортировав библитеку pymystem3.

### Лемматизация

In [127]:
#выделим столбец 'purpose' таблицы и сохраним его данные в переменной purpose_all
purpose_all = pd.Series(data['purpose'])
#объединим все строки столбца в одну строку
purpose_join = ",".join(purpose_all)
#проведем лемматизацию строки
lemmas = m.lemmatize(purpose_join)
#посчитаем количесвто упоминаний лемм
lemmas_count = Counter(lemmas)
display(lemmas_count)

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         ',': 21453,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

**Вывод**

Для проведения лемматизации нам необходимо склеить все значения столбца в одну строку, для этого мы используем метод join(). После мы проводим лемматизацию слов и подсчет упоминаний лемматизированных слов  методом Counter.

### Категоризация данных

In [128]:
#создадим категориальный признак есть дети или нет
data['has_children'] = data['children'] > 0
#создадим сводную таблицу зависимости количества задолжностей от наличия детей. 
children_pivot = data.pivot_table(index=['has_children'], columns='debt', values='family_status_id', aggfunc='count')
display(children_pivot)

debt,0,1
has_children,,
False,13096,1071
True,6617,670


In [129]:
#создадим функцию которая будет выводить категорию семейного положения клиента
def status_group(status_id): 
    if status_id == 0:
        return 'женат / замужем'
    if status_id == 1:
        return 'гражданский брак'
    if status_id == 2:
        return 'вдовец / вдова'
    if status_id == 3:
        return 'в разводе'
    return 'Не женат / не замужем'
#применим метод apply для применения функции ко всем значениям стоблца
data['status_group'] = data['family_status_id'].apply(status_group)
#выведем статистику по группе методом value_counts()
display(data['status_group'].value_counts())
#создадим сводную таблицу из уникальных значений и задолжности
family_pivot = data.pivot_table(index=['status_group'], columns='debt', values='family_status_id', aggfunc='count')
display(family_pivot)


женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: status_group, dtype: int64

debt,0,1
status_group,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931


In [130]:
#создадим функцию которая будет выводить категорию дохода
def income_group(income):   
    if income < 145017:
        return 'низкий'
    if income == 145017:
        return 'средний'
    return 'высокий'
#применим метод apply для применения функции ко всем значениям стоблца
data['income_group'] = data['total_income'].apply(income_group)
#выведем статистику по группе методом value_counts()
display(data['income_group'].value_counts())
#создадим сводную таблицу
income_pivot = data.pivot_table(index=['income_group'], columns='debt', values='family_status_id', aggfunc='count')
display(income_pivot)

низкий     9675
высокий    9675
средний    2104
Name: income_group, dtype: int64

debt,0,1
income_group,,
высокий,8908,767
низкий,8871,804
средний,1934,170


In [131]:
#создадим функцию которая будет выводить цели кредита
def purpose_group(purpose):
    if ('жилье' in purpose) or ('недвижимость' in purpose) or ('жилой' in purpose):
        return 'недвижимость'
    if  'автомобиль' in purpose: 
        return 'автомобиль'
    if  'образование' in purpose:
        return 'образование'
    return 'свадьба'
#применим метод apply для применения функции ко всем значениям стоблца
data['purpose_group'] = data['purpose'].apply(purpose_group)
#выведем статистику по группе методом value_counts()
display(data['purpose_group'].value_counts())
#создадим сводную таблицу
purpose_pivot = data.pivot_table(index=['purpose_group'], columns='debt', values='family_status_id', aggfunc='count')
display(purpose_pivot)

свадьба         12670
недвижимость     5113
образование      2699
автомобиль        972
Name: purpose_group, dtype: int64

debt,0,1
purpose_group,,
автомобиль,882,90
недвижимость,4731,382
образование,2463,236
свадьба,11637,1033


**Вывод**

В данном пункте мы произвели категоризацию данных по необходимым для нас показателям: количеству детей, семейному положению, уровню дохода и целям получения кредита.
Для всех категорий мы создавали сводные таблицы. Данные таблицы в дальнейшем позволят нам произвести анализ показателей и оценить влияние каждой категории на выплаты по кредиту.
Создали категорию наличия детей, которая отражает есть ли у клиента дети или нет. 
Для категоризации по семейному положению, уровню дохода и целям получения кредита мы создали функции, которые группировали значения по условиям, которые мы определили.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [132]:
#создадим сводную таблицу зависимости количества задолжностей от количества детей. 
children_pivot = data.pivot_table(index=['children'], columns='debt', values='family_status_id', aggfunc='count')

#заменим значения NaN в столбце 1
children_pivot[1] = children_pivot[1].fillna(0)

#посчитаем процент клиентов имеющих задолжность для каждой категории стобца children
children_pivot['percent_debt'] = children_pivot[1]/children_pivot[0]*100
display(children_pivot)


debt,0,1,percent_debt
children,,,
0,13096.0,1071.0,8.178070
1,4410.0,445.0,10.090703
2,1858.0,194.0,10.441335
3,303.0,27.0,8.910891
4,37.0,4.0,10.810811
5,9.0,0.0,0.000000


**Вывод**

Исходя из таблицы мы видим, что минимальное количество задолжностей у клиентов у которых нет детей. Так же можно сделать вывод о том, что чем больше количество детей, тем больше задолжностей по выплате кредита, однако есть и исключения. Для семей, в которых 3ое детей, показатель задолжностей, находится примерно на уровне с показателем семей у которых детей нет. это говорит о том, что возможно нам не хватает данных для более точного вывода. тем неменее, мы можем сделать вывод, что количество детей влияет на выплаты по кредиту в срок, бездетные семьи задерживают выплаты реже.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [133]:
#посчитаем процент клиентов имеющих задолжность в зависимости от семейного положения
family_pivot['percent_debt'] = family_pivot[1]/family_pivot[0]*100
display(family_pivot)

debt,0,1,percent_debt
status_group,,,
Не женат / не замужем,2536,274,10.804416
в разводе,1110,85,7.657658
вдовец / вдова,896,63,7.031250
гражданский брак,3763,388,10.310922
женат / замужем,11408,931,8.160940


**Вывод**

Как мы видим из таблицы семейное положение тоже влияет на возврат кредита в срок. Клиенты, которые находятся в разводе или  относятся к категории вдовец / вдова, как правило реже задерживают выплаты. Чаще всего выплаты задерживают клиенты состоящие в гражданском браке, либо относятся к категории: Не женат / не замужем. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [134]:
#посчитаем процент клиентов имеющих задолжность в зависимости от дохода
income_pivot['percent_debt'] = income_pivot[1]/income_pivot[0]*100
display(income_pivot)

debt,0,1,percent_debt
income_group,,,
высокий,8908,767,8.610238
низкий,8871,804,9.063240
средний,1934,170,8.790072


**Вывод**

Процент клиентов имеющих задолжность по кредиту не зависит от ежемесячного дохода клиента.

- Как разные цели кредита влияют на его возврат в срок?

In [135]:
#посчитаем процент клиентов имеющих задолжность в зависимости от цели получения кредита
purpose_pivot['percent_debt'] = purpose_pivot[1]/purpose_pivot[0]*100
display(purpose_pivot)

debt,0,1,percent_debt
purpose_group,,,
автомобиль,882,90,10.204082
недвижимость,4731,382,8.074403
образование,2463,236,9.581811
свадьба,11637,1033,8.876858


**Вывод**

Как мы видим, среди 4 категорий по целям получения кредита, чаще всего вовремя возвращают кредиты с целью кредита на недвижимость. Просрочка платежей чаще всего идет у клиентов, которые берут кредит на автомобиль. Таким образом, мы делаем вывод, что цель кредита влияет на возвращение кредита в срок.


## Шаг 4. Общий вывод

Гипотеза частично подтвердилась.
Количество детей и семейное положение влияет на факт погащения кредита в срок.
Бездетные клиенты чаще всего выплачивают кредит в срок, однако стоит рассмотреть эту гипотезу более тщательно, поскольку есть и исключение в виде клиентов в семьях которых проживет 3ое детей.
Семейное положение также оказывает влияние на своевременность выплат. Клиенты, которые относятся к категории вдовец / вдова, как правило реже задерживают выплаты, в то время как неженатые/незамужние клиенты задерживают выплаты чаще остальных.